In [2]:
import os
import sys
import numpy as np
import matplotlib
import ipympl
import matplotlib
#matplotlib.use('nbagg')
#from matplotlib import style
#style.use('ggplot')
import matplotlib.pyplot as plt

from skimage import feature
from skimage.morphology import reconstruction
from skimage.measure import ransac
from skimage.transform import warp, AffineTransform

from scipy import stats, ndimage
from scipy.misc import factorial as fac

import photutils

from astropy import units
from astropy import stats
from astropy import visualization
from astropy import coordinates
from astropy.table import Table, vstack
from astropy.io import ascii, fits
from astropy.modeling import models, fitting, custom_model

from mmtwfs.wfs import *

%load_ext autoreload
%autoreload 2

In [2]:
bino = WFSFactory(wfs="binospec")

In [3]:
plt.close('all')
bino_file = "/Users/tim/MMT/wfsdat/20180208/wfs_ff_cal_img_2018.0208.074052.fits"
#bino_file = "/Users/tim/MMT/mmtwfs/mmtwfs/data/test_data/wfs_ff_cal_img_2017.1113.111402.fits"
#bino_file = "/Users/tim/MMT/wfsdat/20180209/wfs_ff_cal_img_2018.0209.114054.fits"
#bino_file = "/Users/tim/MMT/wfsdat/20180210/wfs_ff_cal_img_2018.0210.095719.fits"
bino_file = "/Users/tim/MMT/wfsdat/20180210/wfs_ff_cal_img_2018.0210.102615.fits"
results = bino.measure_slopes(bino_file, mode="binospec", plot=True)
results['figures']['slopes'].show()

[ 1.30437828  2.28273345] (1.0008334252415074, 0.996823880806582)


FigureCanvasNbAgg()

In [4]:
refaps = bino.modes['binospec']['reference'].masked_apertures
fig, ax = plt.subplots()
fit = results['grid_fit']
refx = refaps['xcentroid'] * (fit['scale'] + refaps['xcentroid'] * fit['xcoma']) + results['xcen'] +7
refy = refaps['ycentroid'] * (fit['scale'] + refaps['ycentroid'] * fit['ycoma']) + results['ycen'] +11
x, y = results['apertures'].positions.transpose()[0], results['apertures'].positions.transpose()[1]
spacing = 0.5*(results['xspacing'] + results['yspacing'])
ax.scatter(refx, refy)
ax.set_aspect('equal')
ax.scatter(x, y)
ax.set_xlim(0, 512)
ax.set_ylim(0, 512)
fig.show()

results['grid_fit']

FigureCanvasNbAgg()

{'scale': 0.92814492806446891,
 'xcen': 260.70470682928396,
 'xcoma': 9.9999999999999995e-08,
 'ycen': 267.74161501140429,
 'ycoma': 9.9999999999999995e-08}

In [5]:
ref_mask, src_mask = match_apertures(refx, refy, x, y, max_dist=spacing)
ref_mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
       False, False], dtype=bool)

In [6]:
src = np.array((refx[ref_mask], refy[ref_mask])).transpose()
dst = np.array((x[src_mask], y[src_mask])).transpose()

In [7]:
model = AffineTransform()

In [8]:
model_robust, inliers = ransac((src, dst), AffineTransform, min_samples=3, residual_threshold=2, max_trials=100)

In [9]:
model = AffineTransform()
model.estimate(src, dst)

True

In [10]:
src.shape, dst.shape

((95, 2), (95, 2))

In [11]:
model_robust.translation

array([-0.22304539, -0.16205372])

In [12]:
model_robust.scale

(1.0042806266363649, 1.0019221333114974)

In [13]:
inliers

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True, False, False,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
       False, False,  True,  True, False], dtype=bool)

In [3]:
fig, axes = plt.subplots(3, 2)

In [5]:
for a in axes:
    print(a)